In [170]:
import pandas as pd
import numpy as np
import re
from soynlp.utils import DoublespaceLineCorpus
import time

# get the ideas for choosing filtering keywords using soynlp

In [171]:
corpus_fname = 'mycorpus_줄바꿈_적용.txt'   # 이걸로 하세요 여러분 
sentences = DoublespaceLineCorpus(corpus_fname, iter_sent=True)
print('num sentences = %d' % len(sentences))

num sentences = 32786


UsageError: Line magic function `%%time` not found.


In [172]:
%%time

from soynlp.word import WordExtractor

word_extractor = WordExtractor(
    min_frequency=5,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

word_extractor.train(sentences)
words = word_extractor.extract()

training was done. used memory 0.096 Gb
all cohesion probabilities was computed. # words = 16385
all branching entropies was computed # words = 24525
all accessor variety was computed # words = 24525
Wall time: 4.45 s


In [173]:
words

{'겹': Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=0, left_accessor_variety=0, right_accessor_variety=0, leftside_frequency=5, rightside_frequency=0),
 '순': Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=2.697217902811861, right_branching_entropy=1.2177024135383339, left_accessor_variety=34, right_accessor_variety=6, leftside_frequency=496, rightside_frequency=8),
 '멜': Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=-0.0, left_accessor_variety=0, right_accessor_variety=1, leftside_frequency=6, rightside_frequency=0),
 '변': Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=1.8907862716657995, right_branching_entropy=1.8225476701493524, left_accessor_variety=9, right_accessor_variety=7, leftside_frequency=41, rightside_frequency=13),
 '향': Scores(cohesion_forward=0, cohesion_backward=0, left_branching_entropy=3.0527132696909027, right_branch

In [177]:
result = []
for word in words:
    if '빠' in word:
        result.append(word)
        
print(result)

['빠', '빠르', '빠지', '덜빠', '빠른', '빠져', '바빠서', '빠지기', '빠르고', '빠졌는', '배송빠', '빠른배', '안빠져', '머리빠', '빠진다', '빠르면', '빠릅니', '빠르네', '빠져요', '빠른편', '빠지면', '물빠짐', '빠져서', '많이빠', '빠지는', '나빠지', '빠지네', '빠르게', '빠르지', '빠지지', '덜빠진', '덜빠져', '덜빠지', '안빠지', '빠르진', '빠르구', '빠짐이', '빠지고', '빠지는데', '빠지는거', '빠지는것', '머리빠지', '안빠지는', '빠졌는데', '빠르네요', '빠지는게', '빠지네요', '덜빠지는', '덜빠진다', '덜빠져요', '빠릅니다', '배송빠름', '덜빠지고', '빠른배송', '배송도빠', '빠른편이', '배송빠르', '빠지는지', '빠지는건', '빠르구요', '많이빠져', '빠르고요', '머리빠짐', '머리빠짐이', '빠지는거같', '덜빠지는것', '배송빠르네', '배송빠르고', '빠지는것같', '빠른배송굳', '배송도빠르', '빠른배송감', '빠른배송으', '빠지는것같아', '덜빠지는것같', '빠른배송으로', '빠른배송감사', '배송도빠르고', '배송빠르고좋', '빠른배송감사합', '배송빠르고좋아', '배송빠르고좋아요', '빠른배송감사합니', '빠른배송감사합니다']


# extract reviews about hair_loss  

In [181]:
# import dataframe
df = pd.read_excel('excel_02_1018_1단계_CJ_review_peprocessing.xlsx')
len(df)

19896

In [182]:
# remove rows with empty review
df = df.dropna(subset=['review'])
len(df)

19882

In [197]:
# get hair_loss-related comments out of the whole
hair_loss = ['탈모', ' 빠져', ' 빠지', ' 빠진', ' 빠졌', '빠짐', ]
masks = [df['review'].str.contains(string) for string in hair_loss]
combined_masks = np.vstack(masks).any(axis=0)
df_2 = df[combined_masks]
len(df_2)

657

In [198]:
# find positive comments about hair_loss prevention
improved = ['덜', '줄었어요', '안']
masks = [df_2['review'].str.contains(string) for string in improved]
combined_masks = np.vstack(masks).any(axis=0)
df_3 = df_2[combined_masks]
len(df_3)

239

In [199]:
indices_pos = []
indices_neg = []

for k in range(len(df_3)): 
    token = df_3['review'].iloc[k].strip().split()
    index = df_3.index[k]
    
    for i in hair_loss:
        pattern = re.compile(i.strip())
        pattern_2 = re.compile('모르')
        
        for idx, y in enumerate(token):
            m = re.search(pattern, y)
            if m:
                n = re.search(pattern_2, ''.join(token[idx-1:]))
                if n:
                    indices_neg.append(index)
                else:
                    indices_pos.append(index)

In [200]:
# add a column to mark off reviews that express hair_loss improvment
df['탈모'] = np.nan
df['탈모'][indices_pos] = 1
df['탈모'][indices_neg] = 0
df.head()

c:\users\henri\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\henri\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Unnamed: 0,product,review,거품,세정력,자극도,향,탈모
0,0,허벌에센스 핑크로즈 샴푸 490ml,기존에 상품대비 가격이 부담이었는데 할인율이 높아져서 만족하면서 사용하고 있어요...,보통이에요,아주 만족해요,보통이에요,보통이에요,NaN
1,1,허벌에센스 핑크로즈 샴푸 490ml,뽀득뽀득해요 성분 좋다는 샴푸 쓰다가 거품도 너무 안나고 세정력이 떨어져서 바꿔봤는...,거품이 풍성해요,보통이에요,자극없이 순해요,아주 만족해요,NaN
2,2,허벌에센스 핑크로즈 샴푸 490ml,매번쓰던거입니다 거품도풍넝하고맘에아두듭니다,거품이 풍성해요,아주 만족해요,자극없이 순해요,아주 만족해요,NaN
3,3,허벌에센스 핑크로즈 샴푸 490ml,노란색 사용하다가 핑크색이 향이 더 좋을것 같아서 구매했어요,거품이 풍성해요,보통이에요,자극없이 순해요,보통이에요,NaN
4,4,허벌에센스 핑크로즈 샴푸 490ml,비누와 같은 세정력 세장력은 좋은 것 같은데 샴푸 후 머릿결이 조금 뻣뻣한 느낌이 ...,거품이 풍성해요,보통이에요,자극없이 순해요,보통이에요,NaN


In [201]:
df.to_excel('excel_02_1024_1단계_CJ_review_preprocessing_탈모추가_2.xlsx')